In [1]:
import pandas as pd


In [3]:
data = pd.read_csv("/content/tweet_emotions.csv")

In [4]:
df = pd.DataFrame(data)

In [10]:
df = df.drop(["tweet_id"],axis=1)

In [16]:
df = df.rename(columns={"sentiment":"emotion","content":"text"})

In [14]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
  l=[]
  doc = nlp(text)
  for i in doc:
    if(i.is_stop  or i.is_punct):
      continue
    l.append(i.lemma_)
  return " ".join(l)


In [17]:
df["text"] = df["text"].apply(preprocess)

In [ ]:
df

,text,emotion
0,not feel humiliate,sadness
1,feel hopeless damned hopeful care awake,sadness
2,m grab minute post feel greedy wrong,anger
3,feel nostalgic fireplace know property,love
4,feel grouchy,anger
...,...,...
9995,desk job sit ass day long feel paranoid m acti...,fear
9996,feel like confused person lately sigh,fear
9997,feel ve beat word man ground not sleep beneath...,sadness
9998,fully understand feeling beat wound bereft,sadness


In [18]:
num_categories = set(df['emotion'])
df['emotion_num'] = df.emotion.map({
    'anger':5, 'surprise':2, 'sadness':3, 'fear':4, 'joy':1, 'love':0
})

In [31]:
df = df.dropna()

In [32]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(df['text'],df["emotion_num"],test_size=0.1)

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

v = TfidfVectorizer(ngram_range=(1,5))
train_mat = v.fit_transform(X_train)
v.vocabulary_

{'belly9maternity': 16433,
 'tooth': 170844,
 'fairy': 49414,
 'come': 30227,
 'ur': 176707,
 'house': 77781,
 'tonight': 170347,
 'belly9maternity tooth': 16434,
 'tooth fairy': 170872,
 'fairy come': 49415,
 'come ur': 30781,
 'ur house': 176837,
 'house tonight': 77948,
 'belly9maternity tooth fairy': 16435,
 'tooth fairy come': 170873,
 'fairy come ur': 49416,
 'come ur house': 30782,
 'ur house tonight': 176838,
 'belly9maternity tooth fairy come': 16436,
 'tooth fairy come ur': 170874,
 'fairy come ur house': 49417,
 'come ur house tonight': 30783,
 'belly9maternity tooth fairy come ur': 16437,
 'tooth fairy come ur house': 170875,
 'fairy come ur house tonight': 49418,
 'buy': 22993,
 'mac': 102863,
 'buy mac': 23060,
 'happy': 70280,
 'mommy': 111083,
 'day': 36816,
 'mother': 112438,
 'world': 188252,
 'happy mommy': 70602,
 'mommy day': 111093,
 'day mother': 38002,
 'mother world': 113141,
 'happy mommy day': 70606,
 'mommy day mother': 111096,
 'day mother world': 38056,
 '

In [34]:
train_mat.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [35]:
test_mat = v.transform(X_test)

In [36]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(train_mat,Y_train)

MultinomialNB()

In [37]:
pred = model.predict(test_mat)

In [38]:

from sklearn.metrics import classification_report

print(classification_report(Y_test,pred))

              precision    recall  f1-score   support

         0.0       0.81      0.50      0.62       409
         2.0       0.00      0.00      0.00       193
         3.0       0.57      0.97      0.72       517
         5.0       0.00      0.00      0.00        12

    accuracy                           0.62      1131
   macro avg       0.34      0.37      0.33      1131
weighted avg       0.55      0.62      0.55      1131



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


###RandomForest

In [39]:
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier()
model2.fit(train_mat,Y_train)


RandomForestClassifier()

In [40]:
pred2 = model2.predict(test_mat)

In [41]:
from sklearn.metrics import classification_report,precision_recall_curve

print(classification_report(Y_test,pred2))

              precision    recall  f1-score   support

         0.0       0.72      0.67      0.69       409
         2.0       0.34      0.23      0.27       193
         3.0       0.65      0.78      0.71       517
         5.0       0.00      0.00      0.00        12

    accuracy                           0.64      1131
   macro avg       0.43      0.42      0.42      1131
weighted avg       0.61      0.64      0.62      1131



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
t = v.transform(["I’m so mad right now. I can’t talk to you!"])
model2.predict(t)

array([3.])